## Wikipediaの分割
- 固有表現抽出は1,2単語だけ文に対して行うのは難しい．そのため，単語だけで構成される文はパターンマッチで解きたいので，固有表現抽出で解くテキストとパターンマッチで解くテキストの区別を行う

In [1]:
import bs4

In [2]:
# html file
html = '/Users/rikeda/Desktop/Sinra/JP-5 2/HTML/City/31.html'

In [3]:
with open(html, 'r') as f:
    content = f.read()

In [4]:
soup = bs4.BeautifulSoup(content, 'html.parser')

### データ分割
- 実データを見た感じ，以下のルールで分けれそう？
- 文
  - pタグで囲まれているものは本文と思って大丈夫そう
  - 定義中の文：dtタグの中のdd(description, definition)タグは本文と考える
  - class=rellinkも日本語文となっているので文とする
  - class=thumbcaptionも日本語文
  - class=reference-textも文形式だが，ここに固有表現が存在するかは調査する必要がありそう
  - class=thumbimage
- 単語
  - infobox(class=infobox)
  - 表情報: class=wikitable(class='wikitable collapsible'との区別は必要？)
  - dt(description term)
  - class=thumb trightは写真のリンクの名前なのでこっち
  - ul中のliはリストなのでこっち
  - divやclassを利用して，取ってくるリスト，取ってこないリストを先に指定する必要がありそう
  - div style="-moz-column-count:2; -webkit-column-count:2; column-count:2;"はこちらだが，これにも固有表現がつくのか調査をする必要がある
  - table
  
- 単語のルールを作るより，文を抽出するルールを作って，こちらを固有表現抽出で識別し，他をパターンマッチとかで抽出する方が良さそう？
- 文を抽出 -> 抽出した文をHTMLから削除 -> これを繰り返して，余った部分を単語とするのが良さそう？

### 文っぽいやつ

In [5]:
[t.text for t in soup.find_all('p')][:3]

['パリ（仏: Paris[1]、巴里）は、フランス北部、イル＝ド＝フランス地域圏にある都市。フランスの首都であり、イル＝ド＝フランス地域圏の首府である。\n',
 'フランス最大の都市であり、同国の政治、経済、文化などの中心である。ロンドン、ニューヨーク、香港、東京などと並ぶ世界トップクラスの世界都市でもある。行政上では、1コミューン単独で県を構成する特別市であり、ルーヴル美術館を含む1区を中心に、時計回りに20の行政区が並ぶ（エスカルゴと形容される[2]）。\n',
 '市域はティエールの城壁跡に造られた環状高速道路の内側の市街地（面積は86.99km2。参考：東京都・山手線の内側は63km2、ニューヨーク市・マンハッタンは59km2）、および、その外側西部のブローニュの森と外側東部のヴァンセンヌの森を併せた形となっており、面積は105.40\xa0km2。ケスタ地形を呈するパリ盆地のほぼ中央に位置し、市内をセーヌ川が貫く。この川の中州であるシテ島を中心に発達した。市内の地形は比較的平坦であるが、標高は最低でセーヌ川沿いの35メートル、最高でモンマルトルの丘の130メートルである[3]。北緯49度とやや高緯度に位置するが、温かい北大西洋海流と偏西風によって一年を通して比較的温暖となっており、西岸海洋性気候の代表的な都市である。\n']

In [6]:
[t.text for t in soup.find_all('dd')][:3]

[' 凱旋門からテュイルリーまで続く、パリを代表する目抜き通り。パリの歴史軸を構成する。交差するモンテーニュ通りは高級ブランド街。',
 ' フランスの高級ブランドの本店が並ぶ、世界屈指の高級ブランド街。ヴァンドーム広場、コンコルド広場、ルーブル美術館、テュイルリー庭園、に囲まれたパリ中心部の通り。8区内でシャンゼリゼ通りと並走するフォーブール＝サントノレ通りに続いていく。',
 ' パリ発祥の地であり、ノートルダム大聖堂など歴史的建造物も多い。']

In [7]:
[t.text for t in soup.find_all(class_='rellink')][:3]

['詳細は「パリの歴史」を参照', '「フランスの歴史」も参照', '詳細は「フランス革命」を参照']

In [8]:
[t.text for t in soup.find_all(class_='thumbcaption')][:3]

['ランドサット衛星写真', 'パリの行政区', ' エッフェル塔からのパリの360度のパノラマ']

In [9]:
[t.text for t in soup.find_all(class_='reference-text')][:3]

['マスターカードが2014年に公表した統計によると、パリは世界で3番目に外国人旅行者が多く訪れる都市、と報告されている。MasterCard Global Destination Cities Index 2014',
 'フランス語: [paʁi]\xa0(13px 音声ファイル)',
 '「パリのみち」：パリ市街地調査余談（東樋口護 鳥取環境大学教授・副学長）[リンク切れ]']

In [10]:
[t.text for t in soup.find_all(class_='thumbimage')][:3]

[' 左:1795年から1860年までの12の行政区と48のカルティエ\u3000右:1860年以降の20の行政区\n',
 ' 左:1795年から1860年までの12の行政区と48のカルティエ\u3000右:1860年以降の20の行政区\n',
 ' 左:セーヌ川にかかるポン・デ・ザールとポン・ヌフ（東向き）\u3000右:セーヌ川にかかるポンデザール（西向き）\n']

### 単語単位っぽいやつ

In [11]:
# 文に見えるが，表形式のものをテキスト形式で表示しているだけ
[t.text for t in soup.find(class_='infobox').find_all('tr')][:5]

['パリ',
 '\nDrapeau de Paris Flagcoa上：エッフェル塔（前景）とパリ郊外のラ・デファンス地区（背景）下：市旗（左）と紋章（右）',
 '行政',
 '国\nフランスの旗 フランス',
 '地域圏 (Région)\nイル＝ド＝フランス地域圏（地域圏首府）']

In [12]:
[t.text for t in soup.find_all(class_='wikitable')][:3]

['\n歴代市長\n\n\n-\n\n 氏名\n\n 就任\n\n 離任\n\n 政党\n\n 補記（他の資格 等）\n\n\n - \n ジャック・シラク \n 1977年3月20日 \n 1995年5月16日 \n共和国連合\n国民議会議員\n\n\n - \n ジャン・チベリ \n 1995年5月22日 \n 2001年3月24日 \n共和国連合\n国民議会議員\n\n\n - \n ベルトラン・ドラノエ \n 2001年3月25日 \n2014年4月5日\n社会党\n元元老院議員\n\n\n - \n アンヌ・イダルゴ \n 2014年4月5日\n現職\n社会党\n初の女性パリ市長\n\n',
 '\n\n パリ (1971–2000)の気候\n\n\n月\n1月\n2月\n3月\n4月\n5月\n6月\n7月\n8月\n9月\n10月\n11月\n12月\n\n年\n\n\n 最高気温記録 °C (°F)\n\n 16.1 (61)\n\n 21.4 (70.5)\n\n 25.7 (78.3)\n\n 30.2 (86.4)\n\n 34.8 (94.6)\n\n 37.6 (99.7)\n\n 40.4 (104.7)\n\n 39.5 (103.1)\n\n 36.2 (97.2)\n\n 28.4 (83.1)\n\n 21 (70)\n\n 17.1 (62.8)\n\n 40.4 (104.7)\n\n\n 平均最高気温 °C (°F)\n\n 6.9 (44.4)\n\n 8.2 (46.8)\n\n 11.8 (53.2)\n\n 14.7 (58.5)\n\n 19.0 (66.2)\n\n 22.7 (72.9)\n\n 25.2 (77.4)\n\n 25.0 (77)\n\n 20.8 (69.4)\n\n 15.8 (60.4)\n\n 10.4 (50.7)\n\n 7.8 (46)\n\n 15.5 (59.9)\n\n\n 平均最低気温 °C (°F)\n\n 2.5 (36.5)\n\n 2.8 (37)\n\n 5.1 (41.2)\n\n 6.8 (44.2)\n\n 10.5 (50.9)\n\n 13.3 (55.9)\n\n 15.5 (59.9)\n\n 15.4 (59.7)\n\n 12.5 (54.5)\

In [13]:
[t.text for t in soup.find_all('dt')][:5]

[' シャンゼリゼ通り（8区）',
 ' サントノーレ通り（1区）',
 ' シテ島（1区、4区）',
 ' パッシー（16区）',
 ' オペラ界隈（2区、9区）']

In [14]:
[t.text for t in soup.find_all(class_='thumb tright')][:5]

['ファイル:Paris-Landsat001.jpg ランドサット衛星写真',
 'ファイル:Arrondissements-de-Paris.png パリの行政区',
 'ファイル:Thermes-de-Cluny-caldarium.jpg 国立中世博物館に残るクリュニー浴場のカルダリウム',
 'ファイル:Paris in 9 century.jpg 9世紀のシテ島の地図',
 'ファイル:Plan de Paris 1223 BNF07710747.png 1223年のパリ']

- リストの処理．リストを片っ端からとるより，divとかを使って，固有表現が存在する可能性があるところから抽出した方がいい？

In [15]:
[t.text for t in soup.find_all('li')][:10]

['1 概要\n\n1.1 標語\n1.2 風景\n\n',
 '1.1 標語',
 '1.2 風景',
 '2 歴史\n\n2.1 語源\n2.2 古代\n2.3 フランク王国\n2.4 カペー朝\n2.5 ヴァロワ朝\n2.6 ブルボン朝\n2.7 フランス革命\n2.8 19世紀\n2.9 20世紀\n\n',
 '2.1 語源',
 '2.2 古代',
 '2.3 フランク王国',
 '2.4 カペー朝',
 '2.5 ヴァロワ朝',
 '2.6 ブルボン朝']

In [16]:
[t.text for t in soup.find(style='-moz-column-count:2; -webkit-column-count:2; column-count:2;').find_all('li')][:5]

[' オデオン座',
 ' オペラ＝コミック座',
 ' オペラ・ガルニエ（日本で「オペラ座」と呼ぶのはこちら）',
 ' オペラ・バスティーユ',
 ' オランピア劇場']

### 見つかった要素の削除

In [26]:
print(len(soup.get_text()))
elem = soup.find('p')
elem.decompose()
print(len(soup.get_text()))

51142
51006
